In [ ]:
import numpy as np
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv("files\\pred_final.csv")
df

In [ ]:
test_df = df.sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])
test_df

In [ ]:
codes = test_df['Кодзадачи'].unique()
codes_d = {codes[i]: i for i in range(len(codes))}
test_df['Кодзадачи'] =test_df['Кодзадачи'].map(codes_d)
test_df

In [ ]:
test_df['ДатаНачалаЗадачи'] =  pd.to_datetime(test_df['ДатаНачалаЗадачи']).apply(lambda x: x.toordinal())
test_df['ДатаОкончанияЗадачи'] =  pd.to_datetime(test_df['ДатаОкончанияЗадачи']).apply(lambda x: x.toordinal())
test_df

In [ ]:
from tqdm import tqdm
keys = test_df['obj_key'].unique()
test_df2 = test_df.drop(['ДатаОкончанияЗадачи', 'Время на выполнение'], axis=1)
keys.sort()
mm = []
for key in tqdm(keys):
    tt =  test_df2[test_df['obj_key'] == key]
    m_square = []
    codes = tt['Кодзадачи'].unique()
    t_square = []
    for code in codes:
        ttt = tt[(tt['obj_key'] == key) & (tt['Кодзадачи'] == code)]
        code_square = []
        for i, row in ttt.iterrows():
            code_square.append(row.values.tolist())
        t_square.append(code_square)
    m_square.append(t_square)
    mm.append(m_square)

In [ ]:
import numpy as np
mmmm = np.asarray(mm, dtype=object)
mmmm[0]

In [ ]:
test_df.dtypes

In [ ]:
x = test_df.drop(['Время на выполнение','ДатаОкончанияЗадачи', 'date_report', 'Время на выполнение', 'obj_key'], axis=1)
y = test_df['Время на выполнение']
y

In [ ]:
split_index = int(len(test_df) * 0.8)
x_train = x[:split_index]
x_test = x[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [ ]:
x_train['Кодзадачи'].unique()

In [ ]:
cat_features = ['obj_prg', 'obj_subprg', 'Статуспоэкспертизе', 'season', 'Кодзадачи']

In [ ]:
model = CatBoostRegressor(iterations=2000, random_seed=18, loss_function='RMSE', learning_rate=0.05, max_depth=8, early_stopping_rounds=500, eval_metric="MAE")

model.fit(x_train, y_train, use_best_model=True, cat_features=cat_features, eval_set=(x_test, y_test))

In [ ]:
ans = model.predict(x_test)
ans

In [ ]:
x_test

In [ ]:
test_df[split_index:]

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
mean_absolute_error(y_test, ans)